In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
import logging
import penaltyblog as pb
from types import SimpleNamespace
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
import sys
sys.path.append('../src')
from dixon_coles import create_team_ratings, simulate_league, process_simulation_results
from scraping_utils import initialize_webdriver, scroll_page, get_results, get_fixtures, get_table

# Fetch data

In [3]:
driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M7!etejp25/results') # all results
scroll_page(driver)

In [4]:
df = get_results(driver)
df['season'] = '2025'
df['league_division'] = '7'
df

,team_home,team_away,goals_home,goals_away,season,league_division
0,JJ Vepo,ToTe/Arabia,2,3,2025,7
1,FC Lähiö,Valtti/4,2,1,2025,7
2,Tavastia/FC Mojo,FC Puimur,2,1,2025,7
3,Kasiysi/3,LPS/7,3,0,2025,7
4,FC POHU/Sober,Ponnistus/TK,7,1,2025,7
...,...,...,...,...,...,...
161,FC Lähiö,Valtti/TDJ,1,1,2025,7
162,JJ Vepo,LPS/7,1,2,2025,7
163,Ponnistus/TK,Tavastia/FC Mojo,2,6,2025,7
164,Valtti/4,KOPSE/CRPS FC,3,0,2025,7


In [5]:
driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M7!etejp24/results') # all results
scroll_page(driver)

In [6]:
aux = get_results(driver)
aux['season'] = '2024'
aux['league_division'] = '7'
aux

,team_home,team_away,goals_home,goals_away,season,league_division
0,OT-77,Tavastia/FC Mojo,4,4,2024,7
1,FC POHU/Swigu,FC POHU/Sober,8,0,2024,7
2,FC Lähiö,Tavastia/FC Mojo,4,1,2024,7
3,Ponnistus/TK,PiTa/II,2,3,2024,7
4,FC POHU/Swigu,Cosmos,2,0,2024,7
...,...,...,...,...,...,...
189,Valtti/4,FC POHU/Sober,1,0,2024,7
190,Ponnistus/TK,Cosmos,1,4,2024,7
191,ToTe/Arabia,Käpylän Sekunda,1,13,2024,7
192,HeKuLa,Tavastia/FC Mojo,0,3,2024,7


In [7]:
df = pd.concat([df, aux], ignore_index=True)
df

,team_home,team_away,goals_home,goals_away,season,league_division
0,JJ Vepo,ToTe/Arabia,2,3,2025,7
1,FC Lähiö,Valtti/4,2,1,2025,7
2,Tavastia/FC Mojo,FC Puimur,2,1,2025,7
3,Kasiysi/3,LPS/7,3,0,2025,7
4,FC POHU/Sober,Ponnistus/TK,7,1,2025,7
...,...,...,...,...,...,...
355,Valtti/4,FC POHU/Sober,1,0,2024,7
356,Ponnistus/TK,Cosmos,1,4,2024,7
357,ToTe/Arabia,Käpylän Sekunda,1,13,2024,7
358,HeKuLa,Tavastia/FC Mojo,0,3,2024,7


In [8]:
driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M7!etejp23/results') # all results
scroll_page(driver)

In [9]:
aux = get_results(driver)
aux['season'] = '2023'
aux['league_division'] = '7'
aux

,team_home,team_away,goals_home,goals_away,season,league_division
0,FC POHU/Kova Kamppi,Ponnistus/TK,3,2,2023,7
1,Valtti/4,FC Puimur,0,4,2023,7
2,AC StaSi/Europort,Kullervo/Überkleber,2,2,2023,7
3,OT-77,Kullervo,3,0,2023,7
4,LPS/7,FC POHU/Kova Kamppi,3,3,2023,7
...,...,...,...,...,...,...
142,Käpylän Sekunda,FC POHU/Swigu,0,4,2023,7
143,MoPo,PK-35/Äijät,1,2,2023,7
144,FC POHU/Kova Kamppi,FC Puimur,2,2,2023,7
145,Kilo IF/2,Kullervo/Überkleber,2,5,2023,7


In [10]:
df = pd.concat([df, aux], ignore_index=True)
df

,team_home,team_away,goals_home,goals_away,season,league_division
0,JJ Vepo,ToTe/Arabia,2,3,2025,7
1,FC Lähiö,Valtti/4,2,1,2025,7
2,Tavastia/FC Mojo,FC Puimur,2,1,2025,7
3,Kasiysi/3,LPS/7,3,0,2025,7
4,FC POHU/Sober,Ponnistus/TK,7,1,2025,7
...,...,...,...,...,...,...
502,Käpylän Sekunda,FC POHU/Swigu,0,4,2023,7
503,MoPo,PK-35/Äijät,1,2,2023,7
504,FC POHU/Kova Kamppi,FC Puimur,2,2,2023,7
505,Kilo IF/2,Kullervo/Überkleber,2,5,2023,7


In [11]:
driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M6!etejp25/results') # all results
scroll_page(driver)

This is not scraping correctly

In [12]:
aux = get_results(driver)
aux['season'] = '2025'
aux['league_division'] = '6'
aux

,team_home,team_away,goals_home,goals_away,season,league_division
0,Union Plaani/17,Korian Klubi/2,3,3,2025,6
1,HiHi,MyPa/FC,0,0,2025,6
2,Pelikaani,Ares -86,5,1,2025,6
3,Mäntsälän Urheilijat,FC Kapina/2,1,5,2025,6
4,HausPa,JäPS/City,5,1,2025,6
...,...,...,...,...,...,...
633,Vermo Vikings,EPS/2,1,1,2025,6
634,Espoon Sonnit,FC FC,1,0,2025,6
635,Jäntevä/Ukot,HiHi,0,1,2025,6
636,ViSa,FC Etapo,0,0,2025,6


In [13]:
df = pd.concat([df, aux], ignore_index=True)
df

,team_home,team_away,goals_home,goals_away,season,league_division
0,JJ Vepo,ToTe/Arabia,2,3,2025,7
1,FC Lähiö,Valtti/4,2,1,2025,7
2,Tavastia/FC Mojo,FC Puimur,2,1,2025,7
3,Kasiysi/3,LPS/7,3,0,2025,7
4,FC POHU/Sober,Ponnistus/TK,7,1,2025,7
...,...,...,...,...,...,...
1140,Vermo Vikings,EPS/2,1,1,2025,6
1141,Espoon Sonnit,FC FC,1,0,2025,6
1142,Jäntevä/Ukot,HiHi,0,1,2025,6
1143,ViSa,FC Etapo,0,0,2025,6


In [16]:
driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M6!etejp24/results') # all results
scroll_page(driver)

In [17]:
aux = get_results(driver)
aux['season'] = '2024'
aux['league_division'] = '6'
aux

,team_home,team_away,goals_home,goals_away,season,league_division
0,KPPK,FC Mango,2,1,2024,6
1,PaPe,KPPK,2,3,2024,6
2,KPPK,Vietfin FC,0,2,2024,6
3,FC POHU/Gettopojat,FC Mango,1,1,2024,6
4,PaPe,PiTa,2,3,2024,6
...,...,...,...,...,...,...
841,StaPa De Royale,Kulps,10,0,2024,6
842,FC Kapina/2,Ares -86,0,1,2024,6
843,FC City,FC Juvantus,8,3,2024,6
844,HPS/Jägers,Ponnistus/Kermaa,2,1,2024,6


In [18]:
df = pd.concat([df, aux], ignore_index=True)
df

,team_home,team_away,goals_home,goals_away,season,league_division
0,JJ Vepo,ToTe/Arabia,2,3,2025,7
1,FC Lähiö,Valtti/4,2,1,2025,7
2,Tavastia/FC Mojo,FC Puimur,2,1,2025,7
3,Kasiysi/3,LPS/7,3,0,2025,7
4,FC POHU/Sober,Ponnistus/TK,7,1,2025,7
...,...,...,...,...,...,...
1986,StaPa De Royale,Kulps,10,0,2024,6
1987,FC Kapina/2,Ares -86,0,1,2024,6
1988,FC City,FC Juvantus,8,3,2024,6
1989,HPS/Jägers,Ponnistus/Kermaa,2,1,2024,6


In [21]:
driver = initialize_webdriver(f'https://tulospalvelu.palloliitto.fi/category/M6!etejp23/results') # all results
scroll_page(driver)

In [22]:
aux = get_results(driver)
aux['season'] = '2023'
aux['league_division'] = '6'
aux

,team_home,team_away,goals_home,goals_away,season,league_division
0,Wartti/Wild Boars,FC Samba,0,5,2023,6
1,Wartti/Wild Boars,HaPK Edustus/2,2,4,2023,6
2,Pelikaani,Flamingo United,1,4,2023,6
3,FC Samba,HDS/Express,1,2,2023,6
4,Andalus FC,HaPK Edustus/2,2,1,2023,6
...,...,...,...,...,...,...
823,AC StaSi,PK-35/Legendat,3,0,2023,6
824,ToTe/Keparoi,Ponnistus/Kermaa,3,2,2023,6
825,Kullervo/Mosaic,HeMan,0,7,2023,6
826,Ponnistus/Peruskallio,Käpylä Maanantai,0,5,2023,6


In [23]:
df = pd.concat([df, aux], ignore_index=True)
df

,team_home,team_away,goals_home,goals_away,season,league_division
0,JJ Vepo,ToTe/Arabia,2,3,2025,7
1,FC Lähiö,Valtti/4,2,1,2025,7
2,Tavastia/FC Mojo,FC Puimur,2,1,2025,7
3,Kasiysi/3,LPS/7,3,0,2025,7
4,FC POHU/Sober,Ponnistus/TK,7,1,2025,7
...,...,...,...,...,...,...
2814,AC StaSi,PK-35/Legendat,3,0,2023,6
2815,ToTe/Keparoi,Ponnistus/Kermaa,3,2,2023,6
2816,Kullervo/Mosaic,HeMan,0,7,2023,6
2817,Ponnistus/Peruskallio,Käpylä Maanantai,0,5,2023,6


In [24]:
df.to_csv('../data/results/finland_6_7_divisions_results_2023_2025.csv', index=False)

# Data processing

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2819 entries, 0 to 2818
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   team_home        2819 non-null   object
 1   team_away        2819 non-null   object
 2   goals_home       2819 non-null   object
 3   goals_away       2819 non-null   object
 4   season           2819 non-null   object
 5   league_division  2819 non-null   object
dtypes: object(6)
memory usage: 132.3+ KB


In [26]:
df[(df['team_home']=='Trikiinit') | (df['team_away']=='Trikiinit')]

,team_home,team_away,goals_home,goals_away,season,league_division
10,Trikiinit,PK-35/Äijät,6,0,2025,7
12,Valtti/4,Trikiinit,2,5,2025,7
29,MLHF/Reserves,Trikiinit,1,5,2025,7
37,Trikiinit,Kullervo,4,1,2025,7
58,Trikiinit,Cosmos,2,1,2025,7
62,Kullervo,Trikiinit,1,5,2025,7
85,Trikiinit,MoPo,4,1,2025,7
94,Kilo IF/2,Trikiinit,4,6,2025,7
108,Trikiinit,FC POHU/Sober,5,0,2025,7
122,FC Puimur,Trikiinit,1,3,2025,7
